In [1]:
import os
import pandas as pd
import natsort
data_dir =  './korea_data'
data_per = os.listdir(data_dir)

In [21]:
def max_frame(data_per):
    max_frames = 0
    for person in data_per:
        if person == '.DS_Store':
            continue
        words_dir = f'{data_dir}/{person}'
        Id = os.listdir(words_dir)
        for id in Id :
            if id == '.DS_Store':
                    continue
            id_dir = f'{data_dir}/{person}/{id}'
            num = os.listdir(id_dir)
            for n  in num:
                if n == '.DS_Store':
                    continue
                #print("n is", n)
                words_dir_2 = f'{data_dir}/{person}/{id}/{n}'
                frames = os.listdir(words_dir_2)
                #print(len(frames))
                max_frames = max(len(frames), max_frames)
    return max_frames

In [22]:
def concat_x_data(x_data, x_data_):
    if x_data_.shape[0] == max_frame(data_per):
        return np.concatenate([x_data, x_data_], axis = 1) 
    else: 
        pad = max_frame(data_per) - x_data_.shape[1]
        print("pad for ",pad)
        zero_pad = np.zeros((1, pad, 64, 64, 3))
        print(zero_pad.shape)
#         print(x_data_.shape)
        data = np.concatenate([x_data_, zero_pad], axis = 1) #padding
        return data

In [27]:
from tensorflow.keras.preprocessing import image
import numpy as np
x_data = np.empty(5)
for person in natsort.natsorted(data_per):
    if person == '.DS_Store':
        continue
    #phrases_dir = f'{data_dir}/{person}/phrases'
    words_dir = f'{data_dir}/{person}'
    Id = os.listdir(words_dir)
    for sentence in natsort.natsorted(Id) :
        if sentence == '.DS_Store':
            continue
        id_dir = f'{data_dir}/{person}/{sentence}'
        num = os.listdir(id_dir)
        for n  in natsort.natsorted(num):
            if n == '.DS_Store':
                continue
            print("person: {}, sentence: {}, n: {} ".format(person, sentence, n))
            words_dir_2 = f'{data_dir}/{person}/{sentence}/{n}/crop_lip'
            frames = os.listdir(words_dir_2)
            x_data_ = np.empty(4)
            x_data_aug = np.empty(4)
            isFirst = True
            for frame in natsort.natsorted(frames):
                if frame == '.DS_Store':
                    continue
                img_path = f'{words_dir_2}/{frame}'

                # flatten 제거한 frame processing
                img = image.load_img(img_path, target_size=(64, 64))
                img_tensor = image.img_to_array(img) #tensor로 type 변경
                img_tensor /= 255. # scaling 
                img_tensor = np.expand_dims(img_tensor, axis = 0) # (1, 64, 64, 3) 차원
                
                # frame 별 묶기
                if isFirst:
                    x_data_ = img_tensor
                    x_data_aug = DataAugmentation().generate(img)
                    isFisrt = False
                else:
                    x_data_ = np.concatenate([x_data_, img_tensor], axis = 0)
                    img_augmetned = DataAugmentation.generate(img)
                    x_data_aug = np.concatenate([x_data_aug, img_augmetned], axis=0)
                
                print(x_data_.shape)
                
            # 이미지 전체 통합시키기
            x_data_ = np.expand_dims(x_data_, axis = 0)  #[1, frame, 64, 64, 3]
            x_data_aug = np.expand_dims(x_data_aug, axis=0)
            if (n == '00' and sentence == '00' and person == 'A'):
                x_data = concat_x_data(x_data, x_data_)
                y_data = np.array([int(sentence)])
                data_aug = concat_x_data(x_data, x_data_aug)
                x_data = np.concatenate([x_data, data_aug], axis = 0)
                y_data = np.concatenate([y_data, [int(sentence)]])
            else:
                if x_data_.shape[0] == max_frame(data_per):
                    x_data = concat_x_data(x_data, x_data_)
                    y_data = np.concatenate([y_data, [int(sentence)]])
                    x_data = concat_x_data(x_data, x_data_aug)
                    y_data = np.concatenate([y_data, [int(sentence)]])
                else: 
                    data = concat_x_data(x_data, x_data_)
                    x_data = np.concatenate([x_data, data], axis = 0)
                    y_data = np.concatenate([y_data, [int(sentence)]])
                    data_aug = concat_x_data(x_data, x_data_aug)
                    x_data = np.concatenate([x_data, data_aug], axis = 0)
                    y_data = np.concatenate([y_data, [int(sentence)]])
            print(x_data.shape)
            print("y.shape: {}" .format(y_data.shape))

person: A, sentence: 00, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(2, 4, 64, 64, 3)
y.shape: (2,)
person: A, sentence: 00, n: 01 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(4, 4, 64, 64, 3)
y.shape: (4,)
person: A, sentence: 00, n: 02 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 

(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(46, 4, 64, 64, 3)
y.shape: (46,)
person: A, sentence: 07, n: 02 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(48, 4, 64, 64, 3)
y.shape: (48,)
person: A, sentence: 08, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1

(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(96, 4, 64, 64, 3)
y.shape: (96,)
person: B, sentence: 00, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(98, 4, 64, 64, 3)
y.shape: (98,)
person: B, sentence: 00, n: 01 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(100, 4, 64, 64, 3)
y.shape: (100,)
person: B, sentence: 00, n: 02 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(102, 4, 64, 64, 3)
y.shape: (102,)
person: B, sentence: 01, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 6

(148, 4, 64, 64, 3)
y.shape: (148,)
person: B, sentence: 08, n: 02 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(150, 4, 64, 64, 3)
y.shape: (150,)
person: B, sentence: 09, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(152, 4, 64, 64, 3)
y.shape: (152,)
person: B, sentence: 09, n: 01 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64

(198, 4, 64, 64, 3)
y.shape: (198,)
person: C, sentence: 01, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(200, 4, 64, 64, 3)
y.shape: (200,)
person: C, sentence: 01, n: 01 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(202, 4, 64, 64, 3)
y.shape: (202,)
person: C, sentence: 01, n: 02 
(1, 6

(238, 4, 64, 64, 3)
y.shape: (238,)
person: C, sentence: 07, n: 02 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(240, 4, 64, 64, 3)
y.shape: (240,)
person: C, sentence: 08, n: 00 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(242, 4, 64, 64, 3)
y.shape: (242,)
person: C, sentence: 08, n: 01 
(1, 6

(284, 4, 64, 64, 3)
y.shape: (284,)
person: C, sentence: 15, n: 01 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(286, 4, 64, 64, 3)
y.shape: (286,)
person: C, sentence: 15, n: 02 
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
pad for  3
(1, 3, 64, 64, 3)
(288, 4, 64, 64, 3)
y.shape: (288,)


In [37]:
print(y_data[:100])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [10]:
# ref: https://m.blog.naver.com/PostView.nhn?blogId=isu112600&logNo=221582003889&proxyReferer=https:%2F%2Fwww.google.com%2F
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class DataAugmentation:
    @classmethod
    def generate(self, img):
        img_array = image.array_to_img(img)
        samples = np.expand_dims(img_array, axis=0)
        
        # generatior 생성
        datagen = ImageDataGenerator(
            width_shift_range=[-10,10], 
            height_shift_range=0.5, 
            rotation_range=45)
        
        it = datagen.flow(samples, batch_size=1)
        augmented = it.next()
        img_augmented = augmented[0]
        img_augmented /= 255. 
        img_augmented = np.expand_dims(img_augmented, 0)

        return img_augmented